# Migration to and from the United Kingdom by area of destination or origin within the UK by citizenship

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/' \
                  'internationalmigration/datasets/ipsareaofdestinationororiginwithintheukbycitizenship')
scraper

## International Passenger Survey 4.04, area of destination or origin within the UK by citizenship

International Passenger Survey detailed estimates of Long-Term International Migration: Area of destination or origin within the UK by citizenship. UK, - Underlying datasheet 4.

### Distributions

1. International Passenger Survey 4.04, area of destination or origin within the UK by citizenship ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/internationalmigration/datasets/ipsareaofdestinationororiginwithintheukbycitizenship/2017/underlyingdatasheet4.04ipsareaofdestinationororiginwithintheukbycitizenship2017.xls))


In [2]:
tabs = scraper.distribution().as_databaker()

Each tab is of the same form, with "software readable codes":

> The datasheets can be imported directly into suitable software. When importing the datasheets into other software import only rows 8 to 53, starting at column E.

Actually, the admin geo code is also useful, so we'll start at column D

In [3]:
def citizenship_code(s):
    code = pathify(s)
    assert code.startswith('cit-'), code
    code = code[4:]
    assert code.endswith('-est'), code
    code = code[:-4]
    return code.replace('-/-', '-')

def flow_code(s):
    return pathify(s[:s.find(',')])

tidied_sheets = []

for tab in tabs:
    if not tab.name.startswith('Data'):
        continue
    year = int(tab.excel_ref('A2').value[-4:])

    start = tab.excel_ref('D8')
    end = tab.excel_ref('D53')
    admin_geo = start.fill(DOWN) & end.expand(UP)
    codes = admin_geo.shift(RIGHT)
    observations = codes.fill(RIGHT)
    citizenship = start.shift(RIGHT).fill(RIGHT)
    # sheets B, C and D repeat 'All citizenships', 'British' and 'Stateless' from sheet A
    if not tab.name.endswith('A'):
        citizenship = citizenship - citizenship.regex(r'CIT (All|British|Stateless)')
    citizenship_ci = citizenship.regex(r'.*CI\s*$')
    citizenship_est = citizenship - citizenship_ci
    observations_est = observations & citizenship_est.fill(DOWN)
    observations_ci = observations & citizenship_ci.fill(DOWN)
    cs_est = ConversionSegment(observations_est, [
        HDimConst('Year', year),
        HDim(admin_geo, 'Area of Destination or Origin', DIRECTLY, LEFT),
        HDim(codes, 'Migration Flow', DIRECTLY, LEFT),
        HDim(citizenship_est, 'IPS Citizenship', DIRECTLY, ABOVE),
        HDim(observations_ci, 'CI', DIRECTLY, RIGHT),
        HDimConst('Measure Type', 'count'),
        HDimConst('Unit', 'people-thousands')
    ])

    savepreviewhtml(cs_est)
    tidy_sheet = cs_est.topandas()
    tidy_sheet['IPS Citizenship'] = tidy_sheet['IPS Citizenship'].apply(citizenship_code)
    tidy_sheet['Migration Flow'] = tidy_sheet['Migration Flow'].apply(flow_code)
    tidy_sheet = tidy_sheet[pd.isna(tidy_sheet['DATAMARKER'])].copy() # Todo: data markers
    tidy_sheet.drop(columns=['DATAMARKER'], inplace=True)
    tidy_sheet.rename(columns={'OBS': 'Value'}, inplace=True)
    tidy_sheet = tidy_sheet[['Year', 'Area of Destination or Origin', 'Migration Flow', 'IPS Citizenship', 'CI',
                             'Value', 'Measure Type', 'Unit']]
    tidied_sheets.append(tidy_sheet)

In [4]:
tidy = pd.concat(tidied_sheets)

out = Path('out')
out.mkdir(exist_ok=True, parents=True)
tidy.to_csv(out / 'observations.csv', index=False)

In [5]:
scraper.dataset.family = 'migration'
with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())